In [1]:
import yfinance as yf
from binance import Client

In [ ]:
def getStock(tic,start_period):
    ticker = yf.Ticker(tic)
    df = ticker.history(start=start_period)
    return df.values[:,:4],df.index

In [ ]:
def getMonthly(tic,period):
    ticker = yf.Ticker(tic)
    df = ticker.history(period=period)
    df['month'] = [d.month for d in df.index]
    df = df.groupby(['month']).mean()
    return df.values[:,:4],df.index

In [ ]:
def getBTC(tic,start_period):
    #api_key = os.environ['API_KEY']
    #api_secret = os.environ['API_SECRET']
    client = Client()
    klines = client.get_historical_klines(tic, Client.KLINE_INTERVAL_1DAY, start_period, limit=1000)
    df = pd.DataFrame(klines, columns=['OpenTime', 'Open', 'High', 'Low', 'Close', 'Volume', 'CloseTime', 'QuoteVolume',
                                       'NumTrades', 'BuyBaseVolume', 'BuyQuoteVolume', 'Other']
                      ,dtype=np.float32)
    df['DateTime'] = df['OpenTime'].apply(lambda x: datetime.fromtimestamp(x / 1000))
    df = df.set_index('DateTime')
    return df.values[:, 1:5], df.index


In [11]:
import pandas as pd
tic="BTCUSDT"
balance_usdt = client.get_asset_balance(asset='USDT')
df = pd.DataFrame(client.get_ticker())
price = float(df[df.symbol == tic]['lastPrice'].values[0])
fee = client.get_trade_fee(symbol=tic)
print(str(price),fee[0]['makerCommission'])

42778.14 0.001


In [17]:
from binance.enums import *
from decimal import *
fee_pct = float(fee[0]['makerCommission'])
qty = 100.0 * (1 - fee_pct) / price
print("exec buy order at : ", price, round(qty,5), fee_pct, price * qty)
order = client.create_order(
            symbol=tic,
            side=SIDE_BUY,
            type=ORDER_TYPE_LIMIT,
            timeInForce=TIME_IN_FORCE_GTC,
            quantity=round(qty,5),
            price=str(price) )
print(order)

exec buy order at :  42778.14 0.00234 0.001 99.89999999999999
{'symbol': 'BTCUSDT', 'orderId': 8881931245, 'orderListId': -1, 'clientOrderId': '4u2tTTXElLlk0aq5CFRubj', 'transactTime': 1641468835873, 'price': '42778.14000000', 'origQty': '0.00234000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'LIMIT', 'side': 'BUY', 'fills': []}


In [ ]:
stock = 'BTC-USD'
data,ts = getStock(stock,'2017-12-01')

In [ ]:
stock = 'BTCUSDT'
data,ts = getBTC(stock,'2017-12-01')

In [ ]:
from decimal import *
df = pd.DataFrame(client.get_ticker())
price = float(df[df.symbol == 'BTCUSDT']['lastPrice'].values[0])
Decimal(100/price)

In [ ]:
from All_Functions_Master_File import rsi,adder,ma

In [ ]:
my_data = adder(data,5)
my_data = rsi(my_data,14,3,4)
my_data = ma(my_data,20,4,5)


In [ ]:
my_data[1:10,:]

In [ ]:
from datetime import datetime
ts = ts[-1*len(my_data):]
mth = [d.month for d in ts]
yr = [d.year for d in ts]
wd = [d.weekday() for d in ts]
wk = [d.isocalendar()[1] for d in ts]
day = [d.day for d in ts]

In [ ]:
import pandas as pd
df = pd.DataFrame({'Y': my_data[:,5], 'month': mth, 'year': yr, 'weekday': wd, 'weeknum': wk, 'day': day})
dfm = df.groupby(['month']).mean().reset_index()
dfwd = df.groupby(['weekday']).mean().reset_index()
dfwk = df.groupby(['weeknum']).mean().reset_index()
dfymd = df[['Y','year','month']].groupby(['year','month']).mean().reset_index()

In [ ]:
dfymd.merge(dfm[['Y','month']],  on='month',suffixes=['_true','_pred']).sort_values(by=['year','month']).reset_index()

In [ ]:
df_pred = dfymd.merge(dfm[['Y','month']],  on='month',suffixes=['_true','_pred']).sort_values(by=['year','month']).reset_index()
Y_true = df_pred['Y_true'].values
Y_pred = df_pred['Y_pred'].values
Y18 = df_pred[df_pred.year == 2018]['Y_pred']
Y19 = df_pred[df_pred.year == 2019]['Y_pred']
Y20 = df_pred[df_pred.year == 2020]['Y_pred']
Y21 = df_pred[df_pred.year == 2021]['Y_pred']

In [ ]:
import matplotlib.pyplot as plt
# plot the co2 data
fig = plt.figure()
df1 = df_pred[df_pred.year == 2020].reset_index()
df2 = df_pred[df_pred.year == 2021].reset_index()
plt.plot(df1.index,df1['Y_true'], label="2020")
plt.plot(df2.index,df2['Y_true'], label="2021")
plt.ylabel('RSI')
plt.xlabel('time')
plt.legend()

In [ ]:
from sklearn import metrics
print(metrics.mean_absolute_error(Y_true,Y_pred))

In [ ]:
import matplotlib.pyplot as plt
# plot the co2 data
fig = plt.figure()
#plt.plot(df19.index,df19.values)
plt.plot(df_pred.index,Y_true, label="actual value")
plt.plot(df_pred.index,Y_pred)
plt.plot(Y18.index,Y18.values, label="2018 Prediction")
plt.plot(Y19.index,Y19.values, label="2019 Prediction")
plt.plot(Y20.index,Y20.values, label="2020 Prediction")
plt.plot(Y21.index,Y21.values, label="2021 Prediction")
plt.ylabel('RSI')
plt.xlabel('time')
plt.ylim(10, 80)
plt.legend()

In [ ]:
import numpy as np

y = Y_true #my_data[:,5]
x = list(range(len(y)))

# apply fast fourier transform and take absolute values
f=abs(np.fft.fft(y))

# get the list of frequencies
num=np.size(x)
freq = [i / num for i in list(range(num))]

# get the list of spectrums
spectrum=f.real*f.real+f.imag*f.imag
nspectrum=spectrum/spectrum[0]

# plot nspectrum per frequency, with a semilog scale on nspectrum
plt.semilogy(freq,nspectrum)

In [ ]:
len(df[df.year == 2020])

In [ ]:
import statsmodels.api as sm
res = sm.tsa.seasonal_decompose(Y_true,period=6)
resplot = res.plot()


In [ ]:
r = res.resid
r = r[~np.isnan(r)]

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
result = adfuller(r)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
result[1]

In [ ]:
import statsmodels.api as sm
y = my_data[:,5]
res = sm.tsa.seasonal_decompose(y,period=180)
resplot = res.plot()

In [ ]:
r = res.resid
r = r[~np.isnan(r)]

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
result = adfuller(r)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
result[1]

In [ ]:
plt.hist(r, bins = 50)

In [ ]:
from prophet import Prophet

In [ ]:
df

In [ ]:
df['ds'] = df.apply(lambda x: datetime(int(x['year']),int(x['month']),int(x['day'])),axis = 1)
df['y'] = df['Y']

In [ ]:
df

In [ ]:
dfph = df[['ds','y']]
m = Prophet(yearly_seasonality=True)
m.add_seasonality(name='half_year', period=182,fourier_order=5)
m.add_country_holidays(country_name='US')
m.fit(dfph)
future = m.make_future_dataframe(periods=182)
forecast = m.predict(future)
fig1 = m.plot(forecast)

In [ ]:
forecast

In [ ]:
len(forecast.iloc[-100:]['yhat'].values)

In [ ]:
pred_period = 5
pred = forecast.iloc[-1*pred_period:]['yhat'].values

In [ ]:
pred[0]